In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "LoLesports_data/"
SEED = 42

In [3]:
teams_train = pd.read_csv(f"{DATA_PATH}teams_train.csv")
teams_test = pd.read_csv(f"{DATA_PATH}teams_test.csv")

teams_train_target = pd.read_csv(f"{DATA_PATH}teams_train_target.csv")
teams_test_target = pd.read_csv(f"{DATA_PATH}teams_test_target.csv")

In [4]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 111), (9913, 3), (2324, 111), (2324, 3))

In [ ]:
teams_train['teamname'] = teams_train['teamname'].replace(
    "OKSavingsBank Brion", "OKSavingsBank BRION"
)
teams_test['teamname'] = teams_test['teamname'].replace(
    "OKSavingsBank Brion", "OKSavingsBank BRION"
)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
league_locations = {
    "LCK": "Asia/Seoul",
    "LEC": "Europe/Berlin",
    "LCS": "America/Los_Angeles",
    "CBLOL": "America/Sao_Paulo",
    "PCS": "Asia/Taipei",
    "VCS": "Asia/Ho_Chi_Minh",
    "MSI": {
        2022: "Asia/Seoul",
        2023: "Europe/London",
        2024: "Asia/Shanghai"
    },
    "WLDs": {
        2022: "America/Los_Angeles",
        2023: "Asia/Seoul",
        2024: "Europe/Berlin"
    }
}


In [ ]:
import pytz
from datetime import datetime

In [ ]:
for i in range(len(teams_train)):
    date_str = teams_train.loc[i, 'date']

    if len(date_str.split(' ')[1].split(':')[0]) == 1:
        date_str = date_str.replace(' ', ' 0', 1)

    if len(date_str.split(':')) == 2:
        date_str += ':00'

    teams_train.loc[i, 'date'] = date_str

utc = pytz.timezone('UTC')

# 결과 저장할 리스트
local_times = []

# 각 행에 대해 처리
for i in range(len(teams_train)):
    utc_time_str = teams_train.loc[i, 'date']  # UTC 시간 문자열
    league = teams_train.loc[i, 'league']  # 리그 이름

    # UTC 시간을 datetime 객체로 변환
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
    utc_time = utc.localize(utc_time)

    # 연도 추출
    year = utc_time.year

    # 리그의 시간대 가져오기
    if league in league_locations:
        if isinstance(league_locations[league], dict):  # MSI나 WLDs처럼 연도별로 다른 경우
            local_tz = pytz.timezone(league_locations[league].get(year, "UTC"))  # 연도 없으면 기본 UTC
        else:
            local_tz = pytz.timezone(league_locations[league])  # 일반 리그의 시간대
        local_time = utc_time.astimezone(local_tz)  # 로컬 시간대로 변환
        local_time_str = local_time.strftime('%Y-%m-%d %H:%M:%S')  # 문자열로 변환
    else:
        local_time_str = utc_time_str  # 시간대가 없으면 기본 UTC 사용

    # 결과 저장
    local_times.append(local_time_str)

# 변환된 시간 추가
teams_train['local_time'] = local_times


In [ ]:
for i in range(len(teams_test)):
    date_str = teams_test.loc[i, 'date']

    if len(date_str.split(' ')[1].split(':')[0]) == 1:
        date_str = date_str.replace(' ', ' 0', 1)

    if len(date_str.split(':')) == 2:
        date_str += ':00'

    teams_test.loc[i, 'date'] = date_str

utc = pytz.timezone('UTC')

# 결과 저장할 리스트
local_times = []

# 각 행에 대해 처리
for i in range(len(teams_test)):
    utc_time_str = teams_test.loc[i, 'date']  # UTC 시간 문자열
    league = teams_test.loc[i, 'league']  # 리그 이름

    # UTC 시간을 datetime 객체로 변환
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
    utc_time = utc.localize(utc_time)

    # 연도 추출
    year = utc_time.year

    # 리그의 시간대 가져오기
    if league in league_locations:
        if isinstance(league_locations[league], dict):  # MSI나 WLDs처럼 연도별로 다른 경우
            local_tz = pytz.timezone(league_locations[league].get(year, "UTC"))  # 연도 없으면 기본 UTC
        else:
            local_tz = pytz.timezone(league_locations[league])  # 일반 리그의 시간대
        local_time = utc_time.astimezone(local_tz)  # 로컬 시간대로 변환
        local_time_str = local_time.strftime('%Y-%m-%d %H:%M:%S')  # 문자열로 변환
    else:
        local_time_str = utc_time_str  # 시간대가 없으면 기본 UTC 사용

    # 결과 저장
    local_times.append(local_time_str)

# 변환된 시간 추가
teams_test['local_time'] = local_times


In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 112), (9913, 3), (2324, 112), (2324, 3))

In [ ]:
teams_train['local_time'] = pd.to_datetime(teams_train['local_time'])
teams_test['local_time'] = pd.to_datetime(teams_test['local_time'])

# teams_train에 시간 정보 정규화
teams_train['hour'] = teams_train['local_time'].dt.hour
teams_train['hour_sin'] = np.sin(2 * np.pi * teams_train['hour'] / 24)
teams_train['hour_cos'] = np.cos(2 * np.pi * teams_train['hour'] / 24)

# teams_test에 시간 정보 정규화
teams_test['hour'] = teams_test['local_time'].dt.hour
teams_test['hour_sin'] = np.sin(2 * np.pi * teams_test['hour'] / 24)
teams_test['hour_cos'] = np.cos(2 * np.pi * teams_test['hour'] / 24)


In [ ]:
teams_train['year'] = teams_train['local_time'].dt.year
teams_train['month'] = teams_train['local_time'].dt.month
teams_train['day'] = teams_train['local_time'].dt.day
teams_train['time'] = teams_train['local_time'].dt.time

teams_test['year'] = teams_test['local_time'].dt.year
teams_test['month'] = teams_test['local_time'].dt.month
teams_test['day'] = teams_test['local_time'].dt.day
teams_test['time'] = teams_test['local_time'].dt.time


In [ ]:
teams_train.drop(columns=['date'], inplace=True)
teams_test.drop(columns=['date'], inplace=True)
teams_train.drop(columns=['local_time'], inplace=True)
teams_test.drop(columns=['local_time'], inplace=True)

In [ ]:
# teams_train['game_per_day'] = teams_train.groupby(['teamname', 'year', 'month', 'day']).cumcount() + 1
# teams_test['game_per_day'] = teams_test.groupby(['teamname','year', 'month', 'day']).cumcount() + 1

In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 117), (9913, 3), (2324, 117), (2324, 3))

In [ ]:
# 새벽 경기는 전날 저녁으로
#> 그럼 시간 관련된게 너무 많아지는데 이걸 정리해서 드랍을 시켜야하나...... 모델에는 cos sin 넣는거
teams_train['adjusted_hour'] = teams_train['hour']
teams_test['adjusted_hour'] = teams_test['hour']
teams_train.loc[teams_train['hour'] < 6, 'adjusted_hour'] += 24
teams_test.loc[teams_test['hour'] < 6, 'adjusted_hour'] += 24

teams_train['time_period'] = pd.cut(
    teams_train['adjusted_hour'],
    bins=[-1, 12, 18, 30],
    labels=['0', '1', '2']  # 아침: 0, 낮: 1, 밤: 2
)
teams_test['time_period'] = pd.cut(
    teams_test['adjusted_hour'],
    bins=[-1, 12, 18, 30],
    labels=['0', '1', '2']   # 아침: 0, 낮: 1, 밤: 2
)


In [ ]:
teams_train.head(10)

,gameid,league,split,playoffs,game,patch,side,teamname,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,gamelength,result,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,goldspent,gspd,gpr,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,hour,hour_sin,hour_cos,year,month,day,time,adjusted_hour,time_period
0,ESPORTSTMNT01_2700815,LCK,Spring,0,1,12.01,Blue,DRX,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,Aphelios,Sona,Viego,Graves,Ryze,2195,0,5,14,9,0.0,0.0,0.0,0.0,1.0,0.1367,0.5194,0.0,1.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,8.0,9.0,1.0,1.0,10.0,0.0,0.0,2.0,50038,1367.7813,2227.5171,2178.8610,127,3.4715,57,1.5581,55,305,8.3371,63747,39978,1092.7927,60525,0.001902,2.24,945.0,287,33.6765,15121.0,18570.0,330.0,14840.0,18166.0,324.0,281.0,404.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,27198.0,30325.0,511.0,22441.0,28785.0,510.0,4757.0,1540.0,1.0,4.0,7.0,1.0,1.0,1.0,4.0,15,-0.707107,-7.071068e-01,2022,1,12,15:20:00,15,1
1,ESPORTSTMNT01_2700815,LCK,Spring,0,1,12.01,Red,BNK FEARX,Renekton,Lee Sin,Leona,Jayce,Akali,Yuumi,Xin Zhao,Jhin,Syndra,Tryndamere,2195,1,14,5,39,2.0,0.0,0.0,0.0,0.0,0.3827,0.5194,1.0,4.0,1.0,4.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,9.0,8.0,0.0,0.0,0.0,10.0,2.0,0.0,59071,1614.6970,2236.7836,1898.7335,114,3.1162,64,1.7494,43,292,7.9818,67669,43900,1200.0000,60410,-0.001902,-2.24,986.0,204,32.5285,14840.0,18166.0,324.0,15121.0,18570.0,330.0,-281.0,-404.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,22441.0,28785.0,510.0,27198.0,30325.0,511.0,-4757.0,-1540.0,-1.0,1.0,1.0,4.0,4.0,7.0,1.0,15,-0.707107,-7.071068e-01,2022,1,12,15:20:00,15,1
2,ESPORTSTMNT01_2690695,LCK,Spring,0,2,12.01,Blue,DRX,Diana,Caitlyn,Yuumi,Samira,Syndra,Aphelios,Jarvan IV,Thresh,Ryze,Graves,2070,0,7,15,21,0.0,0.0,0.0,0.0,0.0,0.2029,0.6377,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,9.0,0.0,0.0,3.0,1.0,0.0,1.0,66774,1935.4783,2318.4928,2678.0870,117,3.3913,59,1.7101,60,262,7.5942,60674,38182,1106.7246,60660,0.009141,-1.31,994.0,186,34.2029,15495.0,17872.0,318.0,16695.0,19149.0,333.0,-1200.0,-1277.0,-15.0,2.0,5.0,4.0,4.0,5.0,2.0,23612.0,29371.0,528.0,24657.0,30106.0,546.0,-1045.0,-735.0,-18.0,2.0,5.0,4.0,4.0,5.0,2.0,18,-1.000000,-1.836970e-16,2022,1,12,18:02:00,18,1
3,ESPORTSTMNT01_2690695,LCK,Spring,0,2,12.01,Red,BNK FEARX,Renekton,Lee Sin,Twisted Fate,Viktor,LeBlanc,Leona,Xin Zhao,Akali,Ezreal,Tryndamere,2070,1,15,7,31,2.0,0.0,0.0,0.0,1.0,0.4348,0.6377,1.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,9.0,3.0,1.0,1.0,1.0,3.0,1.0,0.0,57616,1670.0290,2938.8116,2685.8551,104,3.0145,60,1.7391,45,253,7.3333,67152,44660,1294.4928,60108,-0.009141,1.31,1051.0,227,37.0435,16695.0,19149.0,333.0,15495.0,17872.0,318.0,1200.0,1277.0,15.0,4.0,5.0,2.0,2.0,5.0,4.0,24657.0,30106.0,546.0,23612.0,29371.0,528.0,1045.0,735.0,18.0,4.0,5.0,2.0,2.0,5.0,4.0,18,-1.000000,-1.836970e-16,2022,1,12,18:02:00,18,1
4,ESPORTSTMNT01_2690705,LCK,Spring,0,1,12.01,Blue,T1,Lee Sin,Ryze,Viktor,LeBlanc,Graves,Karma,Ezreal,Jarvan IV,Gragas,Zoe,2233,1,12,7,26,1.0,1.0,0.0,0.0,1.0,0.3224,0.510

In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 119), (9913, 3), (2324, 119), (2324, 3))

In [ ]:
teams_train.drop(columns=['adjusted_hour'], inplace=True)
teams_test.drop(columns=['adjusted_hour'], inplace=True)

In [ ]:
teams_train['time'] = teams_train['time'].astype(str)
teams_test['time'] = teams_test['time'].astype(str)

teams_train['minute'] = teams_train['time'].str.split(':').str[1].astype(int)
teams_test['minute'] = teams_test['time'].str.split(':').str[1].astype(int)

In [ ]:
teams_train.drop(columns=['time'], inplace=True)
teams_test.drop(columns=['time'], inplace=True)

In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 118), (9913, 3), (2324, 118), (2324, 3))

- 지혁님꺼

In [ ]:
temp_opp_teams = teams_train.groupby("gameid")["teamname"].transform(lambda x: x.iloc[::-1].values).to_frame("opp_teamname")
teams_train = pd.concat([teams_train, temp_opp_teams], axis=1)
temp_opp_teams = teams_test.groupby("gameid")["teamname"].transform(lambda x: x.iloc[::-1].values).to_frame("opp_teamname")
teams_test = pd.concat([teams_test, temp_opp_teams], axis=1)

In [ ]:
cols = ["league", "split", "teamname", "opp_teamname", "ban1", "ban2", "ban3", "ban4", "ban5", "pick1", "pick2", "pick3", "pick4", "pick5"]

teams_train[cols] = teams_train[cols].astype("category")
teams_test[cols] = teams_test[cols].astype("category")

In [ ]:
pre_game_features = [
    "gameid",
    "patch",
    "side",
    "league",
    "teamname",
    "opp_teamname",
    "ban1",
    "ban2",
    "ban3",
    "ban4",
    "ban5",
    "pick1",
    "pick2",
    "pick3",
    "pick4",
    "pick5",
    "year",
    "month",
    "day",
    "hour",
    "hour_sin",
    "hour_cos",
    "time_period",
    "minute"
]

train_ft = teams_train[pre_game_features]
test_ft = teams_test[pre_game_features]

train_ft.shape, test_ft.shape

((9913, 24), (2324, 24))

In [ ]:
stats_columns = [
    "result",
    "gamelength",
    "kills",
    "deaths",
    "assists",
    "firstblood",
    "team kpm",
    "ckpm",
    "firstdragon",
    "firstherald",
    "void_grubs",
    "firstbaron",
    "firsttower",
    "towers",
    "firstmidtower",
    "firsttothreetowers",
    "turretplates",
    "inhibitors",
    "damagetochampions",
    "dpm",
    "damagetakenperminute",
    "damagemitigatedperminute",
    "wardsplaced",
    "wpm",
    "wardskilled",
    "wcpm",
    "controlwardsbought",
    "visionscore",
    "vspm",
]

In [ ]:
# 팀별 최근 승률 계산을 위한 데이터 정렬
temp_train = teams_train.sort_values(["teamname", "year", "month", "day", "hour", "minute"]).reset_index(drop=True)
temp_test = teams_test.sort_values(["teamname", "year", "month", "day", "hour", "minute"]).reset_index(drop=True)

# 팀별 최근 10경기 평균 계산
for col in stats_columns:
    # 승률 계산
    recent10_train = temp_train.groupby("teamname", observed=True)[col].transform(
        lambda x: x.rolling(window=10, min_periods=1).mean().shift(1)
    )
    train_ft = train_ft.assign(**{f"recent10_{col}": recent10_train})

    # 테스트 데이터의 지표 계산을 위해 훈련 데이터와 테스트 데이터 결합
    combined_data = pd.concat([temp_train, temp_test], ignore_index=True).sort_values(
        ["teamname", "year", "month", "day", "hour", "minute"]
    )
    recent10_combined = combined_data.groupby("teamname", observed=True)[col].transform(
        lambda x: x.rolling(window=10, min_periods=1).mean().shift(1)
    )
    combined_data = combined_data.assign(**{f"recent10_{col}": recent10_combined})

    # 테스트 데이터의 지표 업데이트
    recent10_test = combined_data.tail(len(temp_test))[f"recent10_{col}"].values
    test_ft = test_ft.assign(**{f"recent10_{col}": recent10_test})

    # 상대팀 최근 지표 계산
    merged_train = train_ft.merge(
        train_ft[["teamname", "year", "month", "day", "hour", "minute", f"recent10_{col}"]],
        left_on=["opp_teamname", "year", "month", "day", "hour", "minute"],
        right_on=["teamname", "year", "month", "day", "hour", "minute"],
        suffixes=("", "_opp"),
    )
    train_ft = train_ft.assign(
        **{f"opp_recent10_{col}": merged_train[f"recent10_{col}_opp"]}
    )

    merged_test = test_ft.merge(
        combined_data[["teamname", "year", "month", "day", "hour", "minute", f"recent10_{col}"]],
        left_on=["opp_teamname", "year", "month", "day", "hour", "minute"],
        right_on=["teamname", "year", "month", "day", "hour", "minute"],
        suffixes=("", "_opp"),
    )
    test_ft = test_ft.assign(
        **{f"opp_recent10_{col}": merged_test[f"recent10_{col}_opp"]}
    )

    # NaN값 처리 (첫 경기인 경우)
    default_value = 0.5 if col == "result" else 0
    train_ft = train_ft.assign(
        **{
            f"recent10_{col}": train_ft[f"recent10_{col}"].fillna(default_value),
            f"opp_recent10_{col}": train_ft[f"opp_recent10_{col}"].fillna(
                default_value
            ),
        }
    )
    test_ft = test_ft.assign(
        **{
            f"recent10_{col}": test_ft[f"recent10_{col}"].fillna(default_value),
            f"opp_recent10_{col}": test_ft[f"opp_recent10_{col}"].fillna(default_value),
        }
    )

    # 특성 리스트에 새로운 지표 추가
    pre_game_features.extend([f"recent10_{col}", f"opp_recent10_{col}"])

# 입력 데이터 업데이트
train_ft = train_ft[pre_game_features]
test_ft = test_ft[pre_game_features]

train_ft.shape, test_ft.shape

((9913, 82), (2324, 82))

In [ ]:
# 팀별 맞대결 기록을 시간순으로 계산
h2h_records = {}

# 훈련 데이터와 테스트 데이터 결합 후 시간순 정렬
combined_data = pd.concat([teams_train, teams_test], ignore_index=True)
combined_data = combined_data.sort_values(['year', 'month', 'day', 'hour', 'minute'])

# 각 경기마다 이전 맞대결 기록 계산
h2h_winrates = []

for idx, match in combined_data.iterrows():
    team1, team2 = match['teamname'], match['opp_teamname']
    year = match['year']
    key = (team1, team2, year)

    # 현재 시점까지의 맞대결 기록 저장
    if key not in h2h_records:
        h2h_records[key] = {'wins': 0, 'total': 0}
        h2h_winrates.append(0.5)  # 첫 맞대결인 경우 0.5 반환
    else:
        record = h2h_records[key]
        h2h_winrates.append(record['wins'] / record['total'] if record['total'] > 0 else 0.5)

    # 현재 경기 결과 반영
    result = match['result']
    h2h_records[key]['total'] += 1
    if result == 1:
        h2h_records[key]['wins'] += 1

    # 상대팀 관점의 기록도 업데이트
    key_reverse = (team2, team1, year)
    if key_reverse not in h2h_records:
        h2h_records[key_reverse] = {'wins': 0, 'total': 0}
    h2h_records[key_reverse]['total'] += 1
    if result == 0:
        h2h_records[key_reverse]['wins'] += 1

# 계산된 승률을 훈련/테스트 데이터에 할당
train_ft['h2h_winrate'] = h2h_winrates[:len(teams_train)]
test_ft['h2h_winrate'] = h2h_winrates[len(teams_train):]

# 특성 리스트에 h2h_winrate 추가
pre_game_features.append('h2h_winrate')

# 입력 데이터 업데이트
train_ft = train_ft[pre_game_features]
test_ft = test_ft[pre_game_features]

train_ft.shape, test_ft.shape

((9913, 83), (2324, 83))

In [ ]:
# 팀별 리그 승률 기록을 저장할 딕셔너리
league_records = {}
league_winrates = []

# 날짜순으로 정렬
combined_data = pd.concat([teams_train, teams_test], ignore_index=True)
combined_data = combined_data.sort_values(['year', 'month', 'day', 'hour', 'minute'])

# 훈련 데이터에서 팀별 리그 승률 계산
for idx, match in combined_data.iterrows():
    team = match['teamname']
    league = match['league']
    year = match['year']
    key = (team, league, year)

    # 현재 시점까지의 리그 승률 계산
    if key not in league_records:
        league_records[key] = {'wins': 0, 'total': 0}
        league_winrates.append(0.5)  # 첫 경기인 경우 0.5 반환
    else:
        record = league_records[key]
        league_winrates.append(record['wins'] / record['total'] if record['total'] > 0 else 0.5)

    # 현재 경기 결과 반영
    result = match['result']
    league_records[key]['total'] += 1
    if result == 1:
        league_records[key]['wins'] += 1

# 계산된 승률을 훈련/테스트 데이터에 할당
train_ft['league_winrate'] = league_winrates[:len(teams_train)]
test_ft['league_winrate'] = league_winrates[len(teams_train):]

# 특성 리스트에 league_winrate 추가
pre_game_features.append('league_winrate')

# 입력 데이터 업데이트
train_ft = train_ft[pre_game_features]
test_ft = test_ft[pre_game_features]

train_ft.shape, test_ft.shape

((9913, 84), (2324, 84))

In [ ]:
train_ft.head(5)

,gameid,patch,side,league,teamname,opp_teamname,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,year,month,day,hour,hour_sin,hour_cos,time_period,minute,recent10_result,opp_recent10_result,recent10_gamelength,opp_recent10_gamelength,recent10_kills,opp_recent10_kills,recent10_deaths,opp_recent10_deaths,recent10_assists,opp_recent10_assists,recent10_firstblood,opp_recent10_firstblood,recent10_team kpm,opp_recent10_team kpm,recent10_ckpm,opp_recent10_ckpm,recent10_firstdragon,opp_recent10_firstdragon,recent10_firstherald,opp_recent10_firstherald,recent10_void_grubs,opp_recent10_void_grubs,recent10_firstbaron,opp_recent10_firstbaron,recent10_firsttower,opp_recent10_firsttower,recent10_towers,opp_recent10_towers,recent10_firstmidtower,opp_recent10_firstmidtower,recent10_firsttothreetowers,opp_recent10_firsttothreetowers,recent10_turretplates,opp_recent10_turretplates,recent10_inhibitors,opp_recent10_inhibitors,recent10_damagetochampions,opp_recent10_damagetochampions,recent10_dpm,opp_recent10_dpm,recent10_damagetakenperminute,opp_recent10_damagetakenperminute,recent10_damagemitigatedperminute,opp_recent10_damagemitigatedperminute,recent10_wardsplaced,opp_recent10_wardsplaced,recent10_wpm,opp_recent10_wpm,recent10_wardskilled,opp_recent10_wardskilled,recent10_wcpm,opp_recent10_wcpm,recent10_controlwardsbought,opp_recent10_controlwardsbought,recent10_visionscore,opp_recent10_visionscore,recent10_vspm,opp_recent10_vspm,h2h_winrate,league_winrate
0,ESPORTSTMNT01_2700815,12.01,Blue,LCK,DRX,BNK FEARX,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,Aphelios,Sona,Viego,Graves,Ryze,2022,1,12,15,-0.707107,-7.071068e-01,1,20,0.500000,1.000000,0.000000,1595.000000,0.000000,15.000000,0.00,1.0,0.0,29.0,0.00,0.0,0.000000,0.564300,0.000000,0.601900,0.000000,1.000000,0.000000,1.000000,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,11.000000,0.000000,1.000000,0.000000,1.000000,0.0,9.0,0.000000,2.000000,0.000000,44439.000000,0.000000,1671.686500,0.000000,2009.830700,0.000000,1789.203800,0.00,72.0,0.00000,2.70850,0.000000,35.000000,0.000000,1.316600,0.00,30.0,0.000000,207.000000,0.000000,7.786800,0.5,0.5
1,ESPORTSTMNT01_2700815,12.01,Red,LCK,BNK FEARX,DRX,Renekton,Lee Sin,Leona,Jayce,Akali,Yuumi,Xin Zhao,Jhin,Syndra,Tryndamere,2022,1,12,15,-0.707107,-7.071068e-01,1,20,1.000000,0.500000,1595.000000,0.000000,15.000000,0.000000,1.00,0.0,29.0,0.0,0.00,0.0,0.564300,0.000000,0.601900,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,11.000000,0.000000,1.000000,0.000000,1.000000,0.000000,9.0,0.0,2.000000,0.000000,44439.000000,0.000000,1671.686500,0.000000,2009.830700,0.000000,1789.203800,0.000000,72.00,0.0,2.70850,0.00000,35.000000,0.000000,1.316600,0.000000,30.00,0.0,207.000000,0.000000,7.786800,0.000000,1.0,0.5
2,ESPORTSTMNT01_2690695,12.01,Blue,LCK,DRX,BNK FEARX,Diana,Caitlyn,Yuumi,Samira,Syndra,Aphelios,Jarvan IV,Thresh,Ryze,Graves,2022,1,12,18,-1.000000,-1.836970e-16,1,2,1.000000,0.666667,1785.500000,1857.333333,18.500000,14.666667,6.00,8.0,38.5,29.0,0.00,0.0,0.616150,0.480733,0.801950,0.724533,0.500000,0.666667,1.000000,0.666667,0.0,0.0,1.000000,0.666667,1.000000,0.666667,11.000000,8.333333,1.000000,0.666667,1.000000,0.666667,9.5,7.0,2.000000,1.333333,62209.500000,57638.666667,2050.114500,1851.470633,2244.556050,2226.105833,2008.351900,1895.103167,89.00,93.0,2.96355,2.98520,42.000000,45.666667,1.402250,1.464567,38.50,40.0,242.500000,250.333333,8.114050,8.068033,0.0,0.0
3,ESPORTSTMNT01_2690695,12.01,Red,LCK,BNK FEARX,DRX,Renekton,Lee Sin,Twisted Fate,Viktor,LeBlanc,Leona,Xin Zhao,Akali,Ezreal,Tryndamere,2022,1,12,18,-1.000000,-1.836970e-16,1,2,0.666667,1.000000,1857.333333,1785.500000,14.666667,18.500000,8.00,6.0,29.0,38.5,0.00,0.0,0.480733,0.616150,0.724533,0.801950,0.666667,0.500000,0.666667,1.000000,0.0,0.0,0.666667,1.000000,0.666667,1.000000,8.333333,11.000000,0.666667,1.000000,0.666667,1.000000,7.0,9.5,1.333333,2.000000,57638.666667,62209.500000,1851.470633,2050.114500,2226.105833,2244.556050,1895.10316

In [ ]:
teams_train.head(10)

,gameid,league,split,playoffs,game,patch,side,teamname,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,gamelength,result,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,goldspent,gspd,gpr,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,hour,hour_sin,hour_cos,year,month,day,game_per_day,time_period,minute,opp_teamname
0,ESPORTSTMNT01_2700815,LCK,Spring,0,1,12.01,Blue,DRX,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,Aphelios,Sona,Viego,Graves,Ryze,2195,0,5,14,9,0.0,0.0,0.0,0.0,1.0,0.1367,0.5194,0.0,1.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,8.0,9.0,1.0,1.0,10.0,0.0,0.0,2.0,50038,1367.7813,2227.5171,2178.8610,127,3.4715,57,1.5581,55,305,8.3371,63747,39978,1092.7927,60525,0.001902,2.24,945.0,287,33.6765,15121.0,18570.0,330.0,14840.0,18166.0,324.0,281.0,404.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,27198.0,30325.0,511.0,22441.0,28785.0,510.0,4757.0,1540.0,1.0,4.0,7.0,1.0,1.0,1.0,4.0,15,-0.707107,-7.071068e-01,2022,1,12,1,1,20,BNK FEARX
1,ESPORTSTMNT01_2700815,LCK,Spring,0,1,12.01,Red,BNK FEARX,Renekton,Lee Sin,Leona,Jayce,Akali,Yuumi,Xin Zhao,Jhin,Syndra,Tryndamere,2195,1,14,5,39,2.0,0.0,0.0,0.0,0.0,0.3827,0.5194,1.0,4.0,1.0,4.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,9.0,8.0,0.0,0.0,0.0,10.0,2.0,0.0,59071,1614.6970,2236.7836,1898.7335,114,3.1162,64,1.7494,43,292,7.9818,67669,43900,1200.0000,60410,-0.001902,-2.24,986.0,204,32.5285,14840.0,18166.0,324.0,15121.0,18570.0,330.0,-281.0,-404.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,22441.0,28785.0,510.0,27198.0,30325.0,511.0,-4757.0,-1540.0,-1.0,1.0,1.0,4.0,4.0,7.0,1.0,15,-0.707107,-7.071068e-01,2022,1,12,1,1,20,DRX
2,ESPORTSTMNT01_2690695,LCK,Spring,0,2,12.01,Blue,DRX,Diana,Caitlyn,Yuumi,Samira,Syndra,Aphelios,Jarvan IV,Thresh,Ryze,Graves,2070,0,7,15,21,0.0,0.0,0.0,0.0,0.0,0.2029,0.6377,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,9.0,0.0,0.0,3.0,1.0,0.0,1.0,66774,1935.4783,2318.4928,2678.0870,117,3.3913,59,1.7101,60,262,7.5942,60674,38182,1106.7246,60660,0.009141,-1.31,994.0,186,34.2029,15495.0,17872.0,318.0,16695.0,19149.0,333.0,-1200.0,-1277.0,-15.0,2.0,5.0,4.0,4.0,5.0,2.0,23612.0,29371.0,528.0,24657.0,30106.0,546.0,-1045.0,-735.0,-18.0,2.0,5.0,4.0,4.0,5.0,2.0,18,-1.000000,-1.836970e-16,2022,1,12,2,1,2,BNK FEARX
3,ESPORTSTMNT01_2690695,LCK,Spring,0,2,12.01,Red,BNK FEARX,Renekton,Lee Sin,Twisted Fate,Viktor,LeBlanc,Leona,Xin Zhao,Akali,Ezreal,Tryndamere,2070,1,15,7,31,2.0,0.0,0.0,0.0,1.0,0.4348,0.6377,1.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,9.0,3.0,1.0,1.0,1.0,3.0,1.0,0.0,57616,1670.0290,2938.8116,2685.8551,104,3.0145,60,1.7391,45,253,7.3333,67152,44660,1294.4928,60108,-0.009141,1.31,1051.0,227,37.0435,16695.0,19149.0,333.0,15495.0,17872.0,318.0,1200.0,1277.0,15.0,4.0,5.0,2.0,2.0,5.0,4.0,24657.0,30106.0,546.0,23612.0,29371.0,528.0,1045.0,735.0,18.0,4.0,5.0,2.0,2.0,5.0,4.0,18,-1.000000,-1.836970e-16,2022,1,12,2,1,2,DRX
4,ESPORTSTMNT01_2690705,LCK,Spring,0,1,12.01,Blue,T1,Lee Sin,Ryze,Viktor,LeBlanc,Graves,Karma,Ezreal,Jarvan IV,Gragas,Zoe,2233,1,12,7,26,1.0,1.0,0.0,0.0,1.0,

In [ ]:
train_ft.isnull().sum().sum()

0

In [ ]:
test_ft.isnull().sum().sum()

0

In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 119), (9913, 3), (2324, 119), (2324, 3))

In [ ]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 119), (9913, 3), (2324, 119), (2324, 3))

In [ ]:
train_ft.shape, test_ft.shape

((9913, 84), (2324, 84))

In [ ]:
train_ft.to_csv("output/TEST88_train.csv", index=False)
test_ft.to_csv("output/TEST88_test.csv", index=False)